# Lesson 4: Sorting, Aggregation, and Subsets

To jump to the recap, click [here](#recap)

# Initial Setup

Import libraries and initialize variables to pick up where we left off in Lesson 3.

In [ ]:
import pandas as pd

%matplotlib inline

In [ ]:
weather_all = pd.read_csv('data/weather_airport_stations.csv')

# Sorting

- It's often convenient to have our data in a sorted form
- We can also use sorting to answer questions about the extreme (highest / lowest) values in our data

We can sort an entire DataFrame based on the values in a column using the `sort_values` method:

In [ ]:
weather_sorted = weather_all.sort_values('Temperature (C)')
weather_sorted.head()

- The above code does not modify our original DataFrame `weather_all`
- Looking at `weather_sorted`, we can answer questions like:
  - What station(s) and datetimes had the coldest temperatures?
  - What were the weather conditions and other measurements (relative humidity, pressure, etc.) during these coldest temperatures?

- `sort_values` sorts in ascending order (lowest to highest) by default
- We can switch this using the `ascending` keyword argument:

In [ ]:
column = 'Temperature (C)'
weather_all.sort_values(column, ascending=False).head()

# Aggregation

- So far we have looked at statistics for entire columns of a DataFrame
- With aggregation, we can answer questions about sub-groups within columns, such as:
  - What are the mean, minimum, and maximum temperatures at each station in `weather_all`?

We can aggregate data with the `groupby` method chained with an aggregation method (e.g., `mean`, `sum`, `max`, `min`, `count`)
- For example, find the mean values for each station:

In [ ]:
station_means = weather_all.groupby('Station Name').mean()
station_means

For more complex aggregations, there is a `pivot_table` method.

Create a bar chart of the mean temperature at each station:

In [ ]:
temp_means = station_means['Temperature (C)']
temp_means

In [ ]:
temp_means.plot(kind='bar', color='0.5', figsize=(12, 5));

We can use the `sort_values` method on the `temp_means` Series to find out which station had the highest mean temperature:

In [ ]:
temp_means.sort_values(ascending=False)

# Data Subsets

- So far we've been working with entire DataFrames and individual columns
- We can also extract other subsets of a DataFrame:
  - Multiple columns
  - Select rows based on row numbers or row labels
  - Select rows based on a criteria

## Selecting columns of a DataFrame

In [ ]:
weather_all.head()

Get a subset of the DataFrame with only the columns `'Station Name'`, `'Wind Speed (km/hr)'` and `'Temperature (C)'`, in that order:

In [ ]:
columns = ['Station Name', 'Wind Speed (km/hr)', 'Temperature (C)']
winds_temp = weather_all[columns]
winds_temp.head()

With a subset such as `winds_temp`, we can display summaries for just the columns we're interested in. For example, aggregate to find the maximum wind speed and temperature at each station:

In [ ]:
winds_temp.groupby('Station Name').max()

## Selecting rows of a DataFrame
- Selecting rows based on position or row label is actually a fairly complex topic, which we won't cover today
- Let's look at how to select rows based on a criteria
  - Similar to applying a filter in Excel

Where is it snowing?
- Use the string method `contains` on the `'Conditions'` column

In [ ]:
snowing = weather_all['Conditions'].str.contains('Snow')
print(len(snowing))
snowing.head()

- `snowing` is a Boolean Series of length equal to the number of rows of `weather_all`
- The index of `snowing` is equal to the index of `weather_all`
- We can sum the Series to find out how many rows with snowing conditions are in our data:

In [ ]:
num_snowing = snowing.sum()
num_snowing

Alternatively, we could also use `value_counts` to tally up the snowing conditions:

In [ ]:
snowing.value_counts(dropna=False)

We can use `snowing` as a **filter** to extract the rows with snowing weather conditions
- However, our filter can only contain `True` or `False` values
- We need to fill the missing value (`NaN`)
  - Use the `fillna` method to fill the missing with a value of `False`

In [ ]:
snowing_filled = snowing.fillna(False)
snowing_filled.value_counts(dropna=False)

Now we're ready to apply our filter and find out where it's snowing:

In [ ]:
weather_snowing = weather_all[snowing_filled]
weather_snowing

In [ ]:
weather_snowing['Station Name'].unique()

It's snowing at one station&mdash;Iqaluit Airport!

- How many stations have temperatures greater than 20 C?
- Which station had the highest number of hours with temperatures greater than 20 C?

Use a comparison operator to create a filter:

In [ ]:
temp_warm = weather_all['Temperature (C)'] > 20
temp_warm.head()

See how many `True` and `False` are in our filter, and check for missings:

In [ ]:
temp_warm.value_counts(dropna=False)

Temperatures are greater than 20 C in 107 rows of our data, but this doesn't tell us the number of stations, since each station might have multiple rows (multiple hours) with temperatures greater than 20 C.

Use the filter `temp_warm` to extract the rows of `weather_all` which have temperatures greater than 20 C:

In [ ]:
weather_warm = weather_all[temp_warm]
weather_warm.head(3)

The number of stations with temperature greater than 20 C is:

In [ ]:
weather_warm['Station Name'].nunique()

List these stations and the number of hours at each station with temperatures greater than 20 C:

In [ ]:
weather_warm['Station Name'].value_counts()

Winnipeg Richardson Int'l Airport had the highest occurrence (13 hours) of temperatures greater than 20 C

<a id="recap"></a>
# Lesson 4 Recap

### Sorting

Sort a DataFrame based on the values in the column `'Column B'`:
```
df.sort_values('Column B')
```
To sort in descending order, use the keyword argument `ascending=False`


### Aggregation

For basic aggregation operations, use the `groupby` method chained with an aggregation method (e.g., `mean`, `sum`, `max`, `min`, `count`).

For example, to find the mean values for data grouped by `'Column B'`: `
```
df.groupby('Column B').mean()
```

### Subsets

#### Selecting Columns

To select a subset of columns from a DataFrame: 
```
df_sub = df[['Column C', 'Column A', 'Column B']]
```

#### Selecting Rows with a Filter

To select a subset of rows with a filter:
  - Create a filter (Boolean Series)
  - Fill any missings in the filter using the `fillna` method (if necessary)
  - Use the filter to extract the desired rows from the DataFrame

Filter Example 1: string method `contains` with text data
```
snowing = weather_all['Conditions'].str.contains('Snow')
snowing = snowing.fillna(False)
weather_snowing = weather_all[snowing]
```

Filter Example 2: comparison operator with numerical data
```
temp_warm = weather_all['Temperature (C)'] > 20
temp_warm = temp_warm.fillna(False)
weather_warm = weather_all[temp_warm]
```

# Exercise 4

a) What is the fastest wind speed in `weather_all`, and at what station and datetime did it occur? What were the wind direction, temperature, and weather conditions (raining / sunny / etc.) that accompanied this fastest wind speed?

b) How many stations had wind speeds greater than 30 km/hr? Which stations were they?

#### Bonus exercises

c) What were the top three windiest stations, based on their maximum wind speeds? What were the maximum wind speeds at each of these stations?

d) Do the top three stations from (c) change if you rank your stations based on mean wind speed instead of maximum wind speed?

a) What is the fastest wind speed in `weather_all`, and at what station and datetime did it occur? What were the wind direction, temperature, and weather conditions (raining / sunny / etc.) that accompanied this fastest wind speed?

In [ ]:
weather_all.sort_values('Wind Speed (km/hr)', ascending=False).head(1)

The fastest wind speed was 54 km/hr at Iqaluit Airport at 2018-05-22 19:00. Wind direction was NNW, temperature -4 C, and weather conditions partly cloudy.

b) How many stations had wind speeds greater than 30 km/hr? Which stations were they?

In [ ]:
# Create a filter and fill any missings
windy = weather_all['Wind Speed (km/hr)'] > 30
windy = windy.fillna(False)

# Use the filter to extract the desired rows from our DataFrame
weather_windy = weather_all[windy]
weather_windy.head()

In [ ]:
windy_stations = weather_windy['Station Name'].unique()
print(windy_stations)
len(windy_stations)

There were 8 stations with wind speeds exceeding 30 km/hr.

c) What were the top three windiest stations, based on their maximum wind speeds? What were the maximum wind speeds at each of these stations? 

In [ ]:
stations_max = weather_all.groupby('Station Name').max()
stations_max.sort_values('Wind Speed (km/hr)', ascending=False).head(3)

Top 3 windiest stations and their maximum wind speeds:
- 1) Iqaluit: 54 km/hr
- 2) Whitehorse: 42 km/hr
- 3) Quebec Lesage: 39 km/hr

d) Do the top three stations from (c) change if you rank your stations based on mean wind speed instead of maximum wind speed?

In [ ]:
stations_mean = weather_all.groupby('Station Name').mean()
stations_mean.sort_values('Wind Speed (km/hr)', ascending=False).head(3)

Top 3 windiest stations based on mean wind speed:
- 1) Iqaluit: 31.7 km/hr
- 2) St. John's: 23.5
- 3) Whitehorse: 18.9 km/hr